In [1]:
import numpy as np
import function_board as fb
import init_simple_mdp as imdp
import helpers as h

#TODO: double check the loader logic is good [DONE]
    # FGAG loads /Users/rachael/Desktop/darts-thesis/data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_gaussin_prob_grid_custom_no_tokens.pkl
    # Whereas our loader loads player10_e{epsilon}_... 
    # The versions that our loader uses were integrated at a finer level 
    # Loader logic is good

#TODO: check the turn transition probabilities are good (when summing correctly) [DONE]

#TODO: check logic of value function calculation


/Users/rachael/miniconda3/envs/darts/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Sanity Check 

Turn transit probabilities (fast and not) output the same result. 

In [82]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, './original_code/')

import function_evaluate_policy as fep
import function_get_aiming_grid as fgag
import function_solve_dp as fsdp
import helpers as h

data_parameter_dir = fb.data_parameter_dir
result_dir = './result'  
name_pa = 'player{}'.format(10)  


[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

## aiming_grid
num_aiming_location = aiming_grid.shape[0]
prob_normalscore = prob_grid_normalscore_nt
prob_doublescore_dic = {}
for doublescore_index in range(20):
    doublescore = 2*(doublescore_index+1)
    prob_doublescore_dic[doublescore] = np.array(prob_grid_doublescore_nt[:,doublescore_index])
prob_DB = np.array(prob_grid_bullscore_nt[:,1])

## the probability of not bust for each action given score_max=i (score_remain=i+2)
prob_bust_dic = {}
prob_notbust_dic = {}
for score_max in range(60):    
    ## transit to next throw or turn
    prob_notbust = prob_grid_normalscore_nt[:,0:score_max+1].sum(axis=1)
    ## transit to the end of game
    score_remain = score_max + 2
    if (score_remain == fb.score_DB):
        prob_notbust += prob_DB
    elif (score_remain <= 40 and score_remain%2==0):
        prob_notbust += prob_doublescore_dic[score_remain]
    ##
    prob_notbust = np.minimum(np.maximum(prob_notbust, 0),1)
    prob_notbust_dic[score_max] = prob_notbust
    prob_bust_dic[score_max] = 1 - prob_notbust_dic[score_max]

score_state = 10

rd = fsdp.solve_dp_turn(aiming_grid=aiming_grid,prob_grid_normalscore=prob_grid_normalscore_nt,prob_grid_doublescore=prob_grid_doublescore_nt,prob_grid_bullscore=prob_grid_bullscore_nt)

reg = fep.solve_turn_transit_probability(score_state=score_state,state_action=rd['optimal_action_index_dic'][score_state],prob_normalscore=prob_grid_normalscore_nt,prob_doublescore=prob_grid_doublescore_nt,prob_bullscore=prob_grid_bullscore_nt)

fast = fep.solve_turn_transit_probability_fast(score_state=score_state,state_action=rd['optimal_action_index_dic'][score_state],prob_normalscore=prob_grid_normalscore_nt,prob_doublescore=prob_grid_doublescore_nt,prob_bullscore=prob_grid_bullscore_nt,prob_bust_dic=prob_bust_dic)

res_token_yes = h.solve_dp_turn_tokens(9, aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t)

res_token = h.solve_dp_turn_tokens(0, aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t)

[aiming_grid, prob_grid_normalscore_nt_b, prob_grid_singlescore_nt_b, prob_grid_doublescore_nt_b, prob_grid_triplescore_nt_b, prob_grid_bullscore_nt_b] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_nt, prob_grid_singlescore_nt, prob_grid_doublescore_nt, prob_grid_triplescore_nt, prob_grid_bullscore_nt] = fgag.load_aiming_grid(name_pa, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')

res_token = h.solve_dp_turn_tokens(0, aiming_grid, prob_grid_normalscore_nt_b, prob_grid_singlescore_nt_b, prob_grid_doublescore_nt_b, prob_grid_triplescore_nt_b, prob_grid_bullscore_nt_b, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t)


In [208]:
score_state=501

reg = fep.solve_turn_transit_probability(score_state=score_state,state_action=rd['optimal_action_index_dic'][score_state],prob_normalscore=prob_grid_normalscore_nt,prob_doublescore=prob_grid_doublescore_nt,prob_bullscore=prob_grid_bullscore_nt)

fast = fep.solve_turn_transit_probability_fast(score_state=score_state,state_action=rd['optimal_action_index_dic'][score_state],prob_normalscore=prob_grid_normalscore_nt,prob_doublescore=prob_grid_doublescore_nt,prob_bullscore=prob_grid_bullscore_nt,prob_bust_dic=prob_bust_dic)

reg

{'finish': 0,
 'bust': 0,
 'score': array([0.0000e+000, 0.0000e+000, 0.0000e+000, 1.8933e-005, 2.6448e-023, 5.3865e-005, 5.0176e-023, 9.5072e-005, 6.4778e-023, 9.9582e-005, 3.8884e-023, 7.3632e-005, 1.5885e-023, 3.2310e-005, 1.6519e-010, 8.7958e-006, 3.1331e-010, 6.6862e-005, 4.0444e-010, 1.2681e-004, 2.4545e-010, 1.6370e-004,
        6.7006e-004, 9.8216e-005, 1.2709e-003, 4.0106e-005, 1.6405e-003, 4.5567e-016, 9.8428e-004, 3.7216e-016, 4.0193e-004, 7.8706e-005, 3.0121e-010, 7.4643e-005, 6.5686e-012, 6.0961e-005, 1.5775e-003, 3.0186e-009, 1.4960e-003, 6.5828e-011, 1.2218e-003, 7.9046e-003, 2.4025e-009, 7.4961e-003,
        1.3041e-009, 6.1530e-003, 5.3253e-010, 4.5879e-009, 3.8661e-012, 5.1679e-015, 9.2849e-004, 4.8972e-015, 2.5079e-008, 7.7492e-011, 1.9821e-009, 9.3049e-003, 1.6215e-009, 2.0946e-008, 3.9340e-010, 1.9864e-008, 3.1083e-002, 1.6223e-008, 1.2415e-003, 6.0787e-015, 2.3546e-003, 1.1074e-021,
        3.0395e-003, 1.8164e-017, 1.8236e-003, 1.1755e-016, 7.4467e-004, 2.4657e-00

In [209]:
fast

{'finish': 0,
 'bust': 0,
 'score': array([0.0000e+000, 0.0000e+000, 0.0000e+000, 1.8933e-005, 2.6448e-023, 5.3865e-005, 5.0176e-023, 9.5072e-005, 6.4778e-023, 9.9582e-005, 3.8884e-023, 7.3632e-005, 1.5885e-023, 3.2310e-005, 1.6519e-010, 8.7958e-006, 3.1331e-010, 6.6862e-005, 4.0444e-010, 1.2681e-004, 2.4545e-010, 1.6370e-004,
        6.7006e-004, 9.8216e-005, 1.2709e-003, 4.0106e-005, 1.6405e-003, 4.5567e-016, 9.8428e-004, 3.7216e-016, 4.0193e-004, 7.8706e-005, 3.0121e-010, 7.4643e-005, 6.5686e-012, 6.0961e-005, 1.5775e-003, 3.0186e-009, 1.4960e-003, 6.5828e-011, 1.2218e-003, 7.9046e-003, 2.4025e-009, 7.4961e-003,
        1.3041e-009, 6.1530e-003, 5.3253e-010, 4.5879e-009, 3.8661e-012, 5.1679e-015, 9.2849e-004, 4.8972e-015, 2.5079e-008, 7.7492e-011, 1.9821e-009, 9.3049e-003, 1.6215e-009, 2.0946e-008, 3.9340e-010, 1.9864e-008, 3.1083e-002, 1.6223e-008, 1.2415e-003, 6.0787e-015, 2.3546e-003, 1.1074e-021,
        3.0395e-003, 1.8164e-017, 1.8236e-003, 1.1755e-016, 7.4467e-004, 2.4657e-00

In [242]:
h.solve_turn_transit_probability_fast_token(score_state=score_state,state_action=res_token['optimal_action_index_dic'][score_state],available_tokens=0,prob_grid_normalscore_nt=prob_grid_normalscore_nt,prob_grid_doublescore_nt=prob_grid_doublescore_nt,prob_grid_bullscore_nt=prob_grid_bullscore_nt,prob_bust_dic_nt=prob_bust_dic_nt,prob_grid_normalscore_t=prob_grid_normalscore_t,prob_grid_doublescore_t=prob_grid_doublescore_t,prob_grid_bullscore_t=prob_grid_bullscore_t,prob_bust_dic_t=prob_bust_dic_t)

{'finish': 0,
 'bust': array([0.]),
 'score': array([[0.0000e+000, 0.0000e+000, 0.0000e+000, 1.8933e-005, 8.8159e-024, 5.3865e-005, 1.6725e-023, 9.5072e-005, 2.1593e-023, 9.9582e-005, 1.2961e-023, 7.3632e-005, 5.2949e-024, 3.2310e-005, 1.6519e-010, 8.7958e-006, 3.1331e-010, 6.6862e-005, 4.0444e-010, 1.2681e-004, 2.4545e-010, 1.6370e-004,
         6.7006e-004, 9.8216e-005, 1.2709e-003, 4.0106e-005, 1.6405e-003, 4.5564e-016, 9.8428e-004, 3.7212e-016, 4.0193e-004, 7.8706e-005, 3.0121e-010, 7.4643e-005, 6.5686e-012, 6.0961e-005, 1.5775e-003, 3.0186e-009, 1.4960e-003, 6.5828e-011, 1.2218e-003, 7.9046e-003, 2.4025e-009, 7.4961e-003,
         1.3041e-009, 6.1530e-003, 5.3253e-010, 4.5879e-009, 3.8661e-012, 5.1652e-015, 9.2849e-004, 4.8972e-015, 2.5079e-008, 7.7492e-011, 1.9821e-009, 9.3049e-003, 1.6215e-009, 2.0946e-008, 3.9340e-010, 1.9864e-008, 3.1083e-002, 1.6223e-008, 1.2415e-003, 6.0787e-015, 2.3546e-003, 3.6912e-022,
         3.0395e-003, 1.1287e-017, 1.8236e-003, 1.1259e-016, 7.4467e-0

## Win Probability Policy Evaluation

### Evaluate Win Probability Function

In [2]:

def win_probability_policy_evaluation_tokens(value_pa, value_pb, tokens_pa, score_state_pa, score_state_pb, prob_turn_transit_pa, prob_turn_transit_pb):
    """
    Compute the game value (in terms of Player A's winning probability) for a specific turn (score_state_pa, score_state_pb) given Player A and B's policies. 
    Args: 
        value_pa, value_pb: game values for [s_a < score_state_pa, s_b < score_state_pb] are already solved. 
        score_state_pa, score_state_pb: scores for Player A and B at the beginning of this turn
        prob_turn_transit_pa, prob_turn_transit_pb: state transition probability associated with the given policies. 
    
    Returns: 
        value_state_pa: Player A's winning probability when A throws in this turn
        value_state_pb: Player A's winning probability when B throws in this turn
    """
    
    value_win_pa = 1 # A win
    value_win_pb = 0 # A lose

    score_max_pa = min(score_state_pa-2, 3*fb.maxhitscore)
    score_max_pb = min(score_state_pb-2, 3*fb.maxhitscore)

    prob_score_pa = prob_turn_transit_pa['score']
    prob_score_pb = prob_turn_transit_pb['score']
    prob_finish_pa = prob_turn_transit_pa['finish']
    prob_finish_pb = prob_turn_transit_pb['finish']
    prob_zeroscore_pa = prob_turn_transit_pa['bust'][0] + prob_score_pa[0][0]
    prob_zeroscore_pb = prob_turn_transit_pb['bust'][0] + prob_score_pb[0][0]

    possible_tokens_used = prob_turn_transit_pa['bust'].shape[0] - 1

    # sa = sum(sum(prob_turn_transit_pa['score'])) + sum(prob_turn_transit_pa['bust']) + prob_turn_transit_pa['finish']
    # sb = sum(sum(prob_turn_transit_pb['score'])) + sum(prob_turn_transit_pb['bust']) + prob_turn_transit_pb['finish']
    # print(sa,sb)

    # Transit to end
    constant_pa = prob_finish_pa * value_win_pa

    # Only throws startying from Player A's turn, score greater than zero
    constant_pa += np.dot(prob_score_pa[0,1:], value_pb[tokens_pa, score_state_pa-1:score_state_pa-score_max_pa-1:-1, score_state_pb])        

    # Use at least one token:
    if tokens_pa > 0:

        # Go through subsequent scores after using at least one token 
        for i in range(0,possible_tokens_used):

            constant_pa += np.dot(np.flip(prob_score_pa[0:possible_tokens_used+1,:],axis=0)[i], value_pb[tokens_pa-possible_tokens_used:tokens_pa+1,score_state_pa:score_state_pa-score_max_pa-1:-1, score_state_pb][i])        

        # Go bust after using at least one token 
        constant_pa += np.dot(np.flip(prob_turn_transit_pa['bust'][1:possible_tokens_used+1]) , value_pb[tokens_pa-possible_tokens_used+1:tokens_pa+1,score_state_pa,score_state_pb])
        
    constant_pb = prob_finish_pb * value_win_pb #0
    constant_pb += np.dot(prob_score_pb[0][1:], value_pa[tokens_pa,score_state_pa, score_state_pb-1:score_state_pb-score_max_pb-1:-1])

    value_state_pa = (constant_pa+constant_pb*prob_zeroscore_pa)/(1-prob_zeroscore_pa*prob_zeroscore_pb)
    value_state_pb = constant_pb + value_state_pa*prob_zeroscore_pb
    
    return [value_state_pa, value_state_pb]

### Experiment Sandbox

Start by defining which players you want to have play each other. Better to solve for 9 credits and then can look at win probabilities for all combinations. 

In [368]:
data_parameter_dir = fb.data_parameter_dir
result_dir = './result'  
player_num = 10 

epsilon_a = 3
epsilon_b = 1
tokens = 9

name_p = 'player{}'.format(player_num)

[aiming_grid, prob_grid_normalscore_nt_a, prob_grid_singlescore_nt_a, prob_grid_doublescore_nt_a, prob_grid_triplescore_nt_a, prob_grid_bullscore_nt_a] = h.load_aiming_grid(name_p, epsilon=epsilon_a, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_nt_b, prob_grid_singlescore_nt_b, prob_grid_doublescore_nt_b, prob_grid_triplescore_nt_b, prob_grid_bullscore_nt_b] = h.load_aiming_grid(name_p, epsilon=epsilon_b, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')
[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = h.load_aiming_grid('t', data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

prob_grid_doublescore_dic = None
prob_grid_doublescore_dic_t = None

result_dic_a = h.solve_dp_turn_tokens(tokens, aiming_grid, prob_grid_normalscore_nt_a, prob_grid_singlescore_nt_a, prob_grid_doublescore_nt_a, prob_grid_triplescore_nt_a, prob_grid_bullscore_nt_a, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t)
result_dic_b = h.solve_dp_turn_tokens(tokens, aiming_grid, prob_grid_normalscore_nt_b, prob_grid_singlescore_nt_b, prob_grid_doublescore_nt_b, prob_grid_triplescore_nt_b, prob_grid_bullscore_nt_b,prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t)


load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e3_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e1_gaussin_prob_grid_custom_no_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/t_gaussin_prob_grid_custom_tokens.pkl


/Users/rachael/Desktop/darts-thesis/helpers.py:719: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:757: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:1183: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[imdp.throw_num:,score_gained_index] = num_turns_array[imdp.throw_num:,score_gained_index] / prob_notbust_dic_t[score_max][imdp.throw_num:]
/Users/rachael/Desktop/darts-thesis/helpers.py:1210: RuntimeWarning: divide by zero encountered in divide
  value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()
/Users/rachael/Desktop/darts-thesis/helpers.py:1210: RuntimeWarning: invalid value encountered in divide
  value_relerror[rt] = np.abs((state_value_update[rt] - state_value[rt])/state_value_update[rt]).max()


solve prob_policy_transit in 4.524461030960083 seconds
solve dp_turn_policyiter in 39.44088411331177 seconds
[[0.     0.     3.0946 ... 8.938  8.9526 8.9673]
 [0.     0.     1.     ... 7.312  7.3264 7.341 ]
 [0.     0.     1.     ... 6.7576 6.772  6.7866]
 ...
 [0.     0.     1.     ... 4.1559 4.1634 4.1696]
 [0.     0.     1.     ... 3.7995 3.7591 3.7597]
 [0.     0.     1.     ... 3.     3.     3.    ]]


/Users/rachael/Desktop/darts-thesis/helpers.py:719: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:757: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:1190: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic_nt[score_max]
/Users/rachael/Desktop/darts-thesis/helpers.py:1190: RuntimeWarning: overflow encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic_nt[score_max]
/Users/rachael/Desktop/darts-thesis/helpers.py:1177: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[:imdp.throw_num,score_gained_index] = num_turns_array[:imdp.throw_num,score_gained_index] / prob_notbust_dic_nt[score_max][:imdp.throw_num]
/Users/rachael

solve prob_policy_transit in 2.869929075241089 seconds
solve dp_turn_policyiter in 30.102532863616943 seconds
[[0.     0.     1.8856 ... 5.5944 5.5758 5.6116]
 [0.     0.     1.     ... 4.9685 4.9631 4.9765]
 [0.     0.     1.     ... 4.7444 4.7403 4.7516]
 ...
 [0.     0.     1.     ... 3.8072 3.8026 3.783 ]
 [0.     0.     1.     ... 3.5561 3.5447 3.4989]
 [0.     0.     1.     ... 3.     3.     3.    ]]


In [369]:
# Let player a be the weaker player and player b be the stronger player 
tokens_a = 9
game_begin_score_502 = 501+1

# end game state value
value_win_pa = 1 # A win
value_win_pb = 0 # A lose

value_pa = np.zeros((tokens_a+1,502,502))  # player A's winning probability when A throws at state [score_A, score_B]
value_pb = np.zeros((tokens_a+1,502,502))  # player A's winning probability when B throws at state [score_A, score_B]


In [370]:
for tokens_pa in range(tokens_a+1):

    print('---------------------------------------------')
    print(f'Running for {tokens_pa} Tokens')
    print('---------------------------------------------')

    for score_state_pb in range(2,502):

        for score_state_pa in range(2,502):

            prob_turn_transit_pa = result_dic_a['prob_scorestate_transit'][tokens_pa][score_state_pa]
            prob_turn_transit_pb = result_dic_b['prob_scorestate_transit'][0][score_state_pb]

            [value_state_pa, value_state_pb] = win_probability_policy_evaluation_tokens(value_pa, value_pb, tokens_pa, score_state_pa, score_state_pb, prob_turn_transit_pa, prob_turn_transit_pb)

            value_pa[tokens_pa,score_state_pa,score_state_pb] = value_state_pa
            value_pb[tokens_pa,score_state_pa,score_state_pb] = value_state_pb

---------------------------------------------
Running for 0 Tokens
---------------------------------------------
---------------------------------------------
Running for 1 Tokens
---------------------------------------------
---------------------------------------------
Running for 2 Tokens
---------------------------------------------
---------------------------------------------
Running for 3 Tokens
---------------------------------------------
---------------------------------------------
Running for 4 Tokens
---------------------------------------------
---------------------------------------------
Running for 5 Tokens
---------------------------------------------
---------------------------------------------
Running for 6 Tokens
---------------------------------------------
---------------------------------------------
Running for 7 Tokens
---------------------------------------------
---------------------------------------------
Running for 8 Tokens
-----------------------------

In [375]:
value_pa[0]

array([[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.4737, ..., 0.8783, 0.8774, 0.8789],
       ...,
       [0.    , 0.    , 0.0058, ..., 0.0842, 0.083 , 0.0863],
       [0.    , 0.    , 0.0058, ..., 0.084 , 0.0828, 0.0861],
       [0.    , 0.    , 0.0056, ..., 0.0814, 0.0801, 0.0834]])

In [374]:
value_pb[0]

array([[0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
       [0.    , 0.    , 0.2225, ..., 0.8202, 0.8188, 0.8211],
       ...,
       [0.    , 0.    , 0.0027, ..., 0.0289, 0.0282, 0.0298],
       [0.    , 0.    , 0.0027, ..., 0.029 , 0.0283, 0.0299],
       [0.    , 0.    , 0.0026, ..., 0.0277, 0.027 , 0.0286]])

## Generate Win Probability Dataset

In [3]:
data_parameter_dir = fb.data_parameter_dir
result_dir = './result'  
player_num = 10 
name_p = 'player{}'.format(player_num)

[aiming_grid, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t] = h.load_aiming_grid('t', data_parameter_dir=data_parameter_dir, grid_version='custom_tokens')

result_dictionaries = {}

epsilon_list = range(1,10)

for e in epsilon_list: 

    [aiming_grid, prob_grid_normalscore_nt_a, prob_grid_singlescore_nt_a, prob_grid_doublescore_nt_a, prob_grid_triplescore_nt_a, prob_grid_bullscore_nt_a] = h.load_aiming_grid(name_p, epsilon=e, data_parameter_dir=data_parameter_dir, grid_version='custom_no_tokens')

    result_dictionaries[e] = h.solve_dp_turn_tokens(9, aiming_grid, prob_grid_normalscore_nt_a, prob_grid_singlescore_nt_a, prob_grid_doublescore_nt_a, prob_grid_triplescore_nt_a, prob_grid_bullscore_nt_a, prob_grid_normalscore_t, prob_grid_singlescore_t, prob_grid_doublescore_t, prob_grid_triplescore_t, prob_grid_bullscore_t)



load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_tokens/t_gaussin_prob_grid_custom_tokens.pkl
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e1_gaussin_prob_grid_custom_no_tokens.pkl


/Users/rachael/Desktop/darts-thesis/helpers.py:719: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:719: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:757: RuntimeWarning: divide by zero encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:757: RuntimeWarning: overflow encountered in divide
  num_tothrow = num_tothrow / prob_otherstate
/Users/rachael/Desktop/darts-thesis/helpers.py:1190: RuntimeWarning: divide by zero encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_notbust_dic_nt[score_max]
/Users/rachael/Desktop/darts-thesis/helpers.py:1190: RuntimeWarning: overflow encountered in divide
  num_turns_array[:,score_gained_index] = num_turns_array[:,score_gained_index] / prob_no

solve prob_policy_transit in 2.858166217803955 seconds
solve dp_turn_policyiter in 26.079828023910522 seconds
[[0.     0.     1.8856 ... 5.5944 5.5758 5.6116]
 [0.     0.     1.     ... 4.9684 4.9631 4.9765]
 [0.     0.     1.     ... 4.7444 4.7403 4.7516]
 ...
 [0.     0.     1.     ... 3.8073 3.8027 3.8082]
 [0.     0.     1.     ... 3.5561 3.5447 3.5554]
 [0.     0.     1.     ... 3.     3.     3.    ]]
load_pickle from ./data_parameter/player_gaussin_fit/grid_custom_no_tokens/player10_e2_gaussin_prob_grid_custom_no_tokens.pkl
solve prob_policy_transit in 3.738036870956421 seconds
solve dp_turn_policyiter in 28.444954872131348 seconds
[[0.     0.     2.5371 ... 7.405  7.4122 7.4278]
 [0.     0.     1.     ... 6.2669 6.2766 6.2896]
 [0.     0.     1.     ... 5.8664 5.8755 5.8889]
 ...
 [0.     0.     1.     ... 3.9983 3.9972 4.0014]
 [0.     0.     1.     ... 3.7246 3.6864 3.6893]
 [0.     0.     1.     ... 3.     3.     3.    ]]
load_pickle from ./data_parameter/player_gaussin_fit/g

In [12]:
def evaluate_win_probabilities(result_dic_weaker,result_dic_stronger):

    # Let player a be the weaker player and player b be the stronger player 
    tokens_a = 9
    game_begin_score_502 = 501+1

    # end game state value
    value_win_pa = 1 # A win
    value_win_pb = 0 # A lose

    value_pa = np.zeros((tokens_a+1,502,502))  # player A's winning probability when A throws at state [score_A, score_B]
    value_pb = np.zeros((tokens_a+1,502,502))  # player A's winning probability when B throws at state [score_A, score_B]

    result_dic_a = result_dic_weaker
    result_dic_b = result_dic_stronger

    for tokens_pa in range(tokens_a+1):

        for score_state_pb in range(2,502):

            for score_state_pa in range(2,502):

                prob_turn_transit_pa = result_dic_a['prob_scorestate_transit'][tokens_pa][score_state_pa]
                prob_turn_transit_pb = result_dic_b['prob_scorestate_transit'][0][score_state_pb]

                [value_state_pa, value_state_pb] = win_probability_policy_evaluation_tokens(value_pa, value_pb, tokens_pa, score_state_pa, score_state_pb, prob_turn_transit_pa, prob_turn_transit_pb)

                value_pa[tokens_pa,score_state_pa,score_state_pb] = value_state_pa
                value_pb[tokens_pa,score_state_pa,score_state_pb] = value_state_pb

    return value_pa, value_pb      


In [10]:
import pandas as pd

def build_win_probability_dataset(e_stronger, e_weaker, value_weaker, value_stronger):

    df_list = []

    for credits_weaker in range(value_weaker.shape[0]):

        for score_weaker in range(2,502):

            for score_stronger in range(2,502):

                win_probability_weaker_starting = value_weaker[credits_weaker][score_weaker][score_stronger]
                win_probability_stronger_starting = value_stronger[credits_weaker][score_weaker][score_stronger]

                df_list.append([e_weaker,e_stronger,score_weaker,score_stronger,credits_weaker,'weaker',win_probability_weaker_starting])
                df_list.append([e_weaker,e_stronger,score_weaker,score_stronger,credits_weaker,'stronger',win_probability_stronger_starting])

    columns = ['epsilon_weaker','epsilon_stronger','score_weaker','score_stronger','credits_weaker','current_turn','win_probability_weaker']

    return pd.DataFrame(df_list,columns = columns )



In [5]:
epsilon_list=[1,2,3,4,5,6,7,8,9]

In [13]:
columns = ['epsilon_weaker','epsilon_stronger','score_weaker','score_stronger','credits_weaker','current_turn','win_probability_weaker']

for e_stronger in epsilon_list: 

    df = pd.DataFrame(columns=columns)

    print(f"Running for stronger player with epsilon={e_stronger}.")

    for e_weaker in epsilon_list: 

        if e_weaker >= e_stronger: 

            result_dic_weaker = result_dictionaries[e_weaker]
            result_dic_stronger = result_dictionaries[e_stronger]
            value_weaker, value_stronger = evaluate_win_probabilities(result_dic_weaker,result_dic_stronger)

            df = pd.concat([df,build_win_probability_dataset(e_stronger, e_weaker, value_weaker, value_stronger)])
            df = df.reset_index().drop(['index'],axis=1)

    file_title = f'player10_win_probability_results_stronger_e{e_stronger}.csv'
    df.to_csv(file_title)

    del df

Running for stronger player with epsilon=1.
Running for stronger player with epsilon=2.
Running for stronger player with epsilon=3.
Running for stronger player with epsilon=4.
Running for stronger player with epsilon=5.
Running for stronger player with epsilon=6.
Running for stronger player with epsilon=7.
Running for stronger player with epsilon=8.
Running for stronger player with epsilon=9.


In [16]:
columns = ['epsilon_weaker','epsilon_stronger','score_weaker','score_stronger','credits_weaker','current_turn','win_probability_weaker']

df = pd.DataFrame(columns=columns)

for e in epsilon_list:
    print(e)
    df = pd.concat([df,pd.read_csv(f'player10_win_probability_results_stronger_e{e}.csv')])

1
2
3
4
5
6
7
8
9


In [17]:
df.to_csv('player10_win_probability_results.csv')
df

,epsilon_weaker,epsilon_stronger,score_weaker,score_stronger,credits_weaker,current_turn,win_probability_weaker,Unnamed: 0
0,1,1,2,2,0,weaker,0.680422,0.0
1,1,1,2,2,0,stronger,0.319578,1.0
2,1,1,2,3,0,weaker,0.704062,2.0
3,1,1,2,3,0,stronger,0.369911,3.0
4,1,1,2,4,0,weaker,0.589241,4.0
...,...,...,...,...,...,...,...,...
4999995,9,9,501,499,9,stronger,1.000000,4999995.0
4999996,9,9,501,500,9,weaker,1.000000,4999996.0
4999997,9,9,501,500,9,stronger,1.000000,4999997.0
4999998,9,9,501,501,9,weaker,1.000000,4999998.0
